In [3]:
import numpy as np
import pandas as pd

In [4]:
import os
import pandas as pd
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

# Construct the full file paths
movies_path = os.path.join(path, 'tmdb_5000_movies.csv')
credits_path = os.path.join(path, 'tmdb_5000_credits.csv')

# Read the CSV files using the full paths
movies = pd.read_csv(movies_path)
credits = pd.read_csv(credits_path)




100%|██████████| 8.89M/8.89M [00:00<00:00, 48.7MB/s]

Extracting files...


In [5]:
# Store the merged DataFrame in a new variable to avoid overwriting 'movies'
merged_movies = movies.merge(credits, on='title')


In [6]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [7]:
movies = movies[['genres','id','keywords','original_title','overview']]

In [8]:
movies.head()

,genres,id,keywords,original_title,overview
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,A cryptic message from Bond’s past sends him o...
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,Following the death of District Attorney Harve...
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,"John Carter is a war-weary, former military ca..."


In [41]:
movies.isnull().sum()

,0
genres,0
id,0
keywords,0
original_title,0
overview,0


In [10]:
movies.dropna(inplace=True)

In [11]:
movies.duplicated().sum()

0

In [12]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [13]:
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [14]:
movies['genres'] = movies['genres'].apply(convert)

In [15]:
movies.head()

,genres,id,keywords,original_title,overview
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,"[Adventure, Fantasy, Action]",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,"[Action, Adventure, Crime]",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,A cryptic message from Bond’s past sends him o...
3,"[Action, Crime, Drama, Thriller]",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,Following the death of District Attorney Harve...
4,"[Action, Adventure, Science Fiction]",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,"John Carter is a war-weary, former military ca..."


In [16]:
movies['keywords'] = movies['keywords'].apply(convert)

In [17]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [42]:
# Convert lists in 'genres' and 'keywords' columns to strings
movies['genres'] = movies['genres'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
movies['keywords'] = movies['keywords'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

movies['original_title'] = movies['original_title'].apply(lambda x:[i.replace(" ","") for i in x])
movies['original_title'] = movies['original_title'].apply(lambda x: ''.join(x))

In [43]:
movies.head()

,genres,id,keywords,original_title,overview
0,Action Adventure Fantasy ScienceFiction,19995,cultureclash future spacewar spacecolony socie...,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,Adventure Fantasy Action,285,ocean drugabuse exoticisland eastindiatradingc...,PiratesoftheCaribbean:AtWorld'sEnd,"[Captain, Barbossa,, long, believed, to, be, d..."
2,Action Adventure Crime,206647,spy basedonnovel secretagent sequel mi6 britis...,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,Action Crime Drama Thriller,49026,dccomics crimefighter terrorist secretidentity...,TheDarkKnightRises,"[Following, the, death, of, District, Attorney..."
4,Action Adventure ScienceFiction,49529,basedonnovel mars medallion spacetravel prince...,JohnCarter,"[John, Carter, is, a, war-weary,, former, mili..."


In [45]:
for col in ['genres', 'keywords', 'original_title', 'overview']:
    movies[col] = movies[col].str.lower()

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

movies['overview'] = movies['overview'].astype(str) # Converts lists and NaN to string
movies['overview'] = movies['overview'].fillna('')
# Combine relevant columns into a single text data for each movie (if needed)
movie_text_data = (movies['genres'] + " " + movies['keywords'] + " " +
                   movies['original_title'] + " " + movies['overview']).tolist()

# Initialize TF-IDF with stop word removal
tfidf = TfidfVectorizer(stop_words='english')
movie_features = tfidf.fit_transform(movie_text_data)


In [49]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

# Example of stemming on the 'overview' column
movies['overview'] = movies['overview'].apply(lambda x: ' '.join(stemmer.stem(word) for word in x.split()))


In [50]:
movie_text_data = (movies['genres'] + " " + movies['keywords'] + " " +
                   movies['original_title'] + " " + movies['overview']).tolist()

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
movie_features = tfidf.fit_transform(movie_text_data)

In [53]:
movies.to_csv('processed_movies.csv', index=False)

In [54]:
# Check the first few rows to inspect the data structure
print(movies['genres'].head())
print(movies['keywords'].head())

0    action adventure fantasy sciencefiction
1                   adventure fantasy action
2                     action adventure crime
3                action crime drama thriller
4            action adventure sciencefiction
Name: genres, dtype: object
0    cultureclash future spacewar spacecolony socie...
1    ocean drugabuse exoticisland eastindiatradingc...
2    spy basedonnovel secretagent sequel mi6 britis...
3    dccomics crimefighter terrorist secretidentity...
4    basedonnovel mars medallion spacetravel prince...
Name: keywords, dtype: object


In [56]:
# Concatenate columns in memory for each movie as a single list of strings
movie_text_data = (movies['genres'] + " " +
                   movies['keywords'] + " " +
                   movies['original_title'] + " " +
                   movies['overview']).tolist()

In [59]:
# Initialize TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Vectorize the combined text
movie_features = tfidf.fit_transform(movie_text_data)

In [61]:
print(movie_features.shape)

(4800, 14573)


In [62]:
# View the words in the vocabulary
print(tfidf.get_feature_names_out())

['000fathoms' '000leaguesunderthesea' '000movie' ... '친절한금자씨' '태극기휘날리며'
 '해운대']


In [68]:
from sklearn.neighbors import NearestNeighbors

# Initialize the model with cosine distance
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(movie_features)


NearestNeighbors(algorithm='brute', metric='cosine')

In [80]:
def knn_recommend(movie_title, knn_model, movie_features, movies_df, top_n=10):
    # Get the index of the movie
    idx = movies_df[movies_df['original_title'] == movie_title].index[0]

    # Find the top_n most similar movies
    distances, indices = knn_model.kneighbors(movie_features[idx], n_neighbors=top_n + 1)

    # Skip the first neighbor since it's the same movie
    recommended_indices = indices.flatten()[1:]

    # Get the recommended movie titles
    return movies_df['original_title'].iloc[recommended_indices]


In [83]:
# Example: Recommend movies similar to "Inception"
knn_recommendations = knn_recommend("spectre", knn_model, movie_features, movies)
print("Recommended movies:", knn_recommendations.tolist())


Recommended movies: ['quantumofsolace', 'restless', 'skyfall', 'spykids:allthetimeintheworld', 'neversayneveragain', 'red2', 'duplicity', 'ongbak2', '放‧逐', 'theglimmerman']


In [71]:
movies.head()

,genres,id,keywords,original_title,overview
0,action adventure fantasy sciencefiction,19995,cultureclash future spacewar spacecolony socie...,avatar,nan
1,adventure fantasy action,285,ocean drugabuse exoticisland eastindiatradingc...,piratesofthecaribbean:atworld'send,nan
2,action adventure crime,206647,spy basedonnovel secretagent sequel mi6 britis...,spectre,nan
3,action crime drama thriller,49026,dccomics crimefighter terrorist secretidentity...,thedarkknightrises,nan
4,action adventure sciencefiction,49529,basedonnovel mars medallion spacetravel prince...,johncarter,nan
